# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção: (caso o site esteja preparado para evitar esse tipo de utilizaçao do selenium)

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [32]:
import pandas as pd

produtos_df = pd.read_excel('arquivos/buscas.xlsx')
display(produtos_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch 11,3000,3500
1,rtx 3060,zota galax,4000,4500


In [14]:
from selenium import webdriver # Comandar navegador controlado
from selenium.webdriver.chrome.service import Service # responsável por iniciar e parar o chromedriver
from webdriver_manager.chrome import ChromeDriverManager # Gerenciar arquvio ChromeDriver

# Instalar arquivo temporário ChromeDrive a partir da versão atual do Chrome
servico = Service(ChromeDriverManager().install())

# Abrir navegador com tela maximizada
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# Criar navegador
nav = webdriver.Chrome(service=servico, options=options)

In [3]:
# Entrar google shopping
nav.get('https://shopping.google.com/?nord=1')

In [4]:
from selenium.webdriver.common.by import By # Selecionar elementos
from selenium.webdriver.common.keys import Keys # Utilizar comandos do teclado

# Pesquisar produto
nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(produtos_df['Nome'][0])
nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(Keys.ENTER)



In [5]:
# Filtrar Valor

nav.find_element(By.XPATH, '//*[@id="rcnt"]/div[2]/div/div/div[3]/div[2]/div/div/div/span[6]/form/div[1]/div[1]/div/g-text-field/div/div[2]/input').send_keys(str(produtos_df['Preço mínimo'][0]))
nav.find_element(By.XPATH, '//*[@id="rcnt"]/div[2]/div/div/div[3]/div[2]/div/div/div/span[6]/form/div[1]/div[2]/div/g-text-field/div/div[2]/input').send_keys(str(produtos_df['Preço máximo'][0]))
nav.find_element(By.XPATH, '//*[@id="rcnt"]/div[2]/div/div/div[3]/div[2]/div/div/div/span[6]/form/div[1]/div[2]/button').click()

In [35]:
from selenium.webdriver.common.by import By # Selecionar elementos
from selenium.webdriver.common.keys import Keys # Utilizar comandos do teclado

# Criando Function Para Listar Valores Encontrados

def pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max):

    # Entrar google shopping
    nav.get('https://shopping.google.com/?nord=1')

    # Pesquisar produto
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(nome_produto)
    nav.find_element(By.XPATH, '//*[@id="REsRA"]').send_keys(Keys.ENTER)

    # Filtrar Valor min
    nav.find_element(By.NAME, 'lower').send_keys(str(preço_min))
    # Filtrar Valor max
    nav.find_element(By.NAME, 'upper').send_keys(str(preço_max))
    # Aplicar filtro de valor
    nav.find_element(By.NAME, 'upper').send_keys(Keys.ENTER)

    div = '//*[@id="rcnt"]/div[2]/div/div/div[3]/div[2]/div/div/div/span[5]/form/div[1]/div[2]/button'
        
    # Listando elementos com informações do produto pesquisado
    info_produto = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')

    # Listando termos banidos separadamente
    termos_banidos = nomes_banidos.lower()
    termos_banidos = termos_banidos.split(' ')

    # Listando termos do nome do produto separadamente
    termos_produto = nome_produto.lower()
    termos_produto = termos_produto.split(' ')

    # Criando lista com os nome, preço e link do produto
    resultado_pesquisa = []
    for elemento in info_produto:
        
        # Selecionando elemento com nome do produto
        nome_produto = elemento.find_element(By.CLASS_NAME, 'tAxDx').text.lower()

        # Verificando termo banido
        produto_banido = False
        for termo in termos_banidos:
            if termo in nome_produto:
                produto_banido = True
        
        # Verificando se o nome do produto bate com o pesquisado
        produto_aceito = True
        for termo in termos_produto:
            if termo not in nome_produto:
                produto_aceito = False

        # Filtrando nome do produto
        if not produto_banido and produto_aceito:

            # Selecionando elemento com preço do produto
            preço = elemento.find_element(By.CLASS_NAME, 'a8Pemb').text

            # Selecionando elemento com link do produto
            elemento_link_filho = elemento.find_element(By.CLASS_NAME, 'aULzUe')
            elemento_link_pai = elemento_link_filho.find_element(By.XPATH, '..')
            link = elemento_link_pai.get_attribute('href')
            
            resultado_pesquisa.append((nome_produto, preço, link))

    # Ordenando lsta crescentemente a partir do indice 1 da tupla
    resultado_pesquisa.sort(key= lambda x: x[1], reverse=False)
    return resultado_pesquisa


In [36]:
tes = pd.DataFrame()

for linha in produtos_df.index:
    nomes_banidos = produtos_df.loc[linha, 'Termos banidos']
    nome_produto = produtos_df.loc[linha, 'Nome']
    preço_max = produtos_df.loc[linha, 'Preço máximo']
    preço_min = produtos_df.loc[linha, 'Preço mínimo']

    teste = pesquisa_google(nav, nome_produto, nomes_banidos, preço_min, preço_max)
    teste_df = pd.DataFrame(teste, columns=['produto', 'preço', 'link'])
    tes = pd.concat([tes, teste_df])

display(tes)

,produto,preço,link
0,apple iphone 12 branco 64gb,"R$ 3.024,00",https://www.google.com/url?url=https://doji.co...
1,apple iphone 12 preto 64gb,"R$ 3.024,00",https://www.google.com/url?url=https://doji.co...
2,usado: iphone 12 64gb preto bom - trocafone,"R$ 3.109,00",https://www.google.com/url?url=https://www.col...
3,usado: iphone 12 64gb azul muito bom - trocafo...,"R$ 3.140,72",https://www.google.com/url?url=https://www.mag...
4,"apple iphone 12 with facetime, 64gb, 5g, dual ...","R$ 3.155,23 + impostos",https://www.google.com/url?url=https://global....
...,...,...,...
43,pc gamer intel i5 10400f 16gb (rtx 3060 8gb) s...,"R$ 4.459,89",https://www.google.com/url?url=https://www.sho...
44,workstation intermédia a intel core i5 10400f ...,"R$ 4.468,50",https://www.google.com/url?url=https://www.4of...
45,gaming pc i7-6700 480gb ssd 1 tb hdd 16 gb ram...,"R$ 4.485,64",https://www.google.com/url?url=https://properm...
46,pc gamer amd ryzen 5 5500 16gb (rtx 3060 8gb) ...,"R$ 4.489,90",https://www.google.com/url?url=https://www.sho...


In [5]:
print(produtos_df['Preço mínimo'][0])

3000
